<a href="https://colab.research.google.com/github/Maflibu/-Ai-Data-preprocessing-/blob/main/time%20study%20model%20on%20colab%20Ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define paths to your data
base_dir = '/content/drive/MyDrive/timestudy_cnn1'
train_dir = os.path.join(base_dir, 'images_labelled')
video_path = '/content/drive/MyDrive/timestudy_cnn/videos/assembly.mp4'
output_csv = '/content/drive/MyDrive/timestudy_cnn/output/activity_results.csv'

# Define image parameters
img_height, img_width = 100, 100
batch_size = 32

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(6, activation='softmax')  # 6 classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Image data augmentation
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10  # adjust as needed
)



Mounted at /content/drive
Found 226 images belonging to 6 classes.
Epoch 1/10
7/7 [==============================] - 104s 13s/step - loss: 1.6326 - accuracy: 0.2732
Epoch 2/10
7/7 [==============================] - 1s 122ms/step - loss: 1.6046 - accuracy: 0.2938
Epoch 3/10
7/7 [==============================] - 1s 121ms/step - loss: 1.4922 - accuracy: 0.3247
Epoch 4/10
7/7 [==============================] - 1s 154ms/step - loss: 1.3794 - accuracy: 0.3036
Epoch 5/10
7/7 [==============================] - 1s 186ms/step - loss: 1.1517 - accuracy: 0.4948
Epoch 6/10
7/7 [==============================] - 1s 158ms/step - loss: 0.9698 - accuracy: 0.6186
Epoch 7/10
7/7 [==============================] - 1s 129ms/step - loss: 0.7406 - accuracy: 0.6856
Epoch 8/10
7/7 [==============================] - 1s 123ms/step - loss: 0.5729 - accuracy: 0.7732
Epoch 9/10
7/7 [==============================] - 1s 121ms/step - loss: 0.5882 - accuracy: 0.7526
Epoch 10/10
7/7 [==============================] - 

In [2]:
# Save the trained model
model_save_path = '/content/drive/MyDrive/timestudy_cnn1/trained_model'
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)
model.save(os.path.join(model_save_path, 'my_model.h5'))

print("Model saved successfully.")

Model saved successfully.


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
# Load the saved model
from tensorflow.keras.models import load_model

saved_model_path = '/content/drive/MyDrive/timestudy_cnn1/trained_model/my_model.h5'
model = load_model(saved_model_path)

# Define paths to your data
video_path = '/content/drive/MyDrive/timestudy_cnn1/videos/assembly.mp4'
output_csv = '/content/drive/MyDrive/timestudy_cnn1/output/activity_results.csv'

# Function to predict activities in a video clip
def predict_activities(video_path, model):
    cap = cv2.VideoCapture(video_path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    timestamps = []
    activities = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        resized_frame = cv2.resize(frame, (img_width, img_height))
        resized_frame = np.expand_dims(resized_frame, axis=0)
        prediction = model.predict(resized_frame)
        predicted_class = np.argmax(prediction)
        timestamps.append(cap.get(cv2.CAP_PROP_POS_MSEC))
        activities.append(predicted_class)
    cap.release()
    return timestamps, activities

# Predict activities
timestamps, activities = predict_activities(video_path, model)

# Write results to CSV
df = pd.DataFrame({'Timestamp': timestamps, 'Activity': activities})
df.to_csv(output_csv, index=False)

print("Results saved to", output_csv)

1/1 [==============================] - 0s 30ms/step
Results saved to /content/drive/MyDrive/timestudy_cnn1/output/activity_results.csv
